In [8]:
import sys

sys.path.append(r"C:\Users\Admin\Desktop\ai")

from tone_shift.features.feature_extraction import extract_features
from tone_shift.synth.synthesis import synthesize_from_params, make_wav
import os
import numpy as np

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class TimbreDecoder(nn.Module):
    def __init__(
        self,
        in_dim: int,         
        gru_hidden: int = 256,
        gru_layers: int = 2,
        mlp_hidden: int = 256,
        harmonics_dim: int = 60,   
        noise_dim: int = 65        
    ):
        super().__init__()

        self.gru = nn.GRU(
            input_size=in_dim,
            hidden_size=gru_hidden,
            num_layers=gru_layers,
            batch_first=True,
            bidirectional=False,
        )

        self.mlp = nn.Sequential(
            nn.Linear(gru_hidden, mlp_hidden),
            nn.LayerNorm(mlp_hidden),
            nn.ReLU(),
            nn.Linear(mlp_hidden, mlp_hidden),
            nn.ReLU(),
        )

        self.harmonics_head = nn.Linear(mlp_hidden, harmonics_dim)
        self.noise_head = nn.Linear(mlp_hidden, noise_dim)
        self.gain_head = nn.Linear(mlp_hidden, 1)

    def forward(self, x, h0=None):
        gru_out, h_n = self.gru(x, h0)  

        z = self.mlp(gru_out)            

        harmonics = self.harmonics_head(z)
        noise = self.noise_head(z)
        gain = self.gain_head(z)

        harmonics = F.softplus(harmonics)
        harmonics_sum = harmonics.sum(dim=-1, keepdim=True) + 1e-8
        harmonics = harmonics / harmonics_sum

        noise = F.softplus(noise)

        gain = torch.sigmoid(gain)

        return harmonics, noise, gain, h_n


In [10]:
class MultiScaleSpectrogramLoss(nn.Module):
    def __init__(self,
                 fft_sizes=(2048, 1024, 512),
                 hop_sizes=(512, 256, 128),
                 win_lengths=(2048, 1024, 512),
                 mag_power: float = 1.0):
        super().__init__()
        assert len(fft_sizes) == len(hop_sizes) == len(win_lengths)
        self.fft_sizes = fft_sizes
        self.hop_sizes = hop_sizes
        self.win_lengths = win_lengths
        self.mag_power = mag_power

    def stft_mag(self, x, fft_size, hop_size, win_length):
        window = torch.hann_window(win_length, device=x.device)
        X = torch.stft(
            x,
            n_fft=fft_size,
            hop_length=hop_size,
            win_length=win_length,
            window=window,
            center=True,
            pad_mode="reflect",
            return_complex=True,
        )
        mag = torch.abs(X) ** self.mag_power
        return mag

    def forward(self, y_hat, y):
        loss = 0.0
        for fft_size, hop, win in zip(self.fft_sizes, self.hop_sizes, self.win_lengths):
            Y_hat = self.stft_mag(y_hat, fft_size, hop, win)
            Y = self.stft_mag(y, fft_size, hop, win)
            loss += F.l1_loss(torch.log1p(Y_hat), torch.log1p(Y))
        return loss / len(self.fft_sizes)


In [ ]:


import numpy as np
import torch

SR = 16000
HOP_LENGTH = 512
FRAME_RATE = SR / HOP_LENGTH

def prepare_features_for_decoder(wav_path: str):
    """
    Wraps extract_features and builds tensors:
      feats_t: [1, T, 3]  (f0_log, loudness, onset)
      f0_t:    [1, T]
      audio_t: [1, N]
    """
    out = extract_features(wav_path)

    # Handle both 4-return and 5-return versions:
    if len(out) == 4:
        f0, loudness, onset, y = out
    elif len(out) == 5:
        f0, loudness, onset, y, _ = out   # e.g. voicing or extra info
    else:
        raise RuntimeError(f"Unexpected number of outputs from extract_features: {len(out)}")

    # ----- F0 -----
    f0 = np.asarray(f0, dtype=np.float32)
    f0 = np.nan_to_num(f0, nan=0.0)
    f0_hz = np.clip(f0, 1.0, 3000.0).astype(np.float32)
    f0_log = np.log2(f0_hz / 440.0).astype(np.float32)

    # ----- Loudness / Onset -----
    loudness = np.asarray(loudness, dtype=np.float32)
    onset = np.asarray(onset, dtype=np.float32)

    loudness = np.nan_to_num(loudness, nan=0.0, posinf=0.0, neginf=0.0)
    onset = np.nan_to_num(onset, nan=0.0, posinf=0.0, neginf=0.0)

    # ----- Align lengths -----
    T = min(len(f0_log), len(loudness), len(onset))
    f0_hz = f0_hz[:T]
    f0_log = f0_log[:T]
    loudness = loudness[:T]
    onset = onset[:T]

    # ----- Stack features [f0_log, loudness, onset] -----
    feats = np.stack([f0_log, loudness, onset], axis=-1).astype(np.float32)
    feats = np.nan_to_num(feats, nan=0.0, posinf=0.0, neginf=0.0)

    # ----- Torch tensors -----
    feats_t = torch.from_numpy(feats).unsqueeze(0)                # [1, T, 3]
    f0_t = torch.from_numpy(f0_hz).unsqueeze(0)                   # [1, T]
    audio_t = torch.from_numpy(y.astype(np.float32)).unsqueeze(0) # [1, N]

    frame_rate = FRAME_RATE
    return feats_t, f0_t, audio_t, frame_rate



In [ ]:
with torch.no_grad():
    harmonics, noise, gain, _ = decoder(feats_t.to(device))
    params = {
        "harmonic_amps": harmonics,
        "noise_amps": noise,
        "gain": gain,
        "optional": {}
    }
    y_hat = synthesize_from_params(params, f0_t.to(device), sr=SR, frame_rate=frame_rate)

make_wav(y_hat, SR) 
print("y_hat shape:", y_hat.shape)
print("min/max:", y_hat.min().item(), y_hat.max().item())
print("has NaN:", torch.isnan(y_hat).any().item())


y_hat shape: torch.Size([1, 64512])
min/max: -1.0 1.0
has NaN: False


Organ Timbre Transfer Train

In [12]:
import os
import glob
import shutil

PROJECT_ROOT = r"C:\Users\Admin\Desktop\ai\tone_shift"
DATA_ROOT = os.path.join(PROJECT_ROOT, "data")

SPLITS = ["nsynth-train", "nsynth-test"] 

ORGAN_DIR = os.path.join(DATA_ROOT, "nsynth-organ", "audio")
os.makedirs(ORGAN_DIR, exist_ok=True)

total = 0
for split in SPLITS:
    src_audio = os.path.join(DATA_ROOT, split, "audio")
    pattern = os.path.join(src_audio, "organ_*.wav")
    organ_files = glob.glob(pattern)

    print(f"{split}: found {len(organ_files)} organ files")

    for src_path in organ_files:
        dst_path = os.path.join(ORGAN_DIR, os.path.basename(src_path))
        shutil.copy2(src_path, dst_path)
        total += 1

print(f"Total organ wavs copied: {total}")
print(f"Saved in: {ORGAN_DIR}")


nsynth-train: found 34477 organ files


KeyboardInterrupt: 

In [25]:
import os
import glob
import random
from torch.utils.data import Dataset

class OrganDataset(Dataset):
    def __init__(self, audio_dir, max_files=None, seed=42):
        paths = sorted(glob.glob(os.path.join(audio_dir, "*.wav")))
        if not paths:
            raise RuntimeError(f"No wav files found in {audio_dir}")

        # Randomly subsample if max_files is set
        if max_files is not None and len(paths) > max_files:
            random.seed(seed)
            paths = random.sample(paths, max_files)

        self.paths = paths
        print(f"Using {len(self.paths)} organ files from {audio_dir}")

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        wav_path = self.paths[idx]
        feats_t, f0_t, audio_t, frame_rate = prepare_features_for_decoder(wav_path)

        feats = feats_t.squeeze(0)   # [T, 3]
        f0    = f0_t.squeeze(0)      # [T]
        audio = audio_t.squeeze(0)   # [N]

        return {
            "feats": feats,
            "f0": f0,
            "audio": audio,
            "frame_rate": frame_rate,
            "path": wav_path,
        }

ORGAN_AUDIO_DIR = os.path.join(DATA_ROOT, "nsynth-organ", "audio")

max_files = 200  # try 100 / 200 / 500 etc.
organ_dataset = OrganDataset(ORGAN_AUDIO_DIR, max_files=max_files)

from torch.utils.data import DataLoader

organ_loader = DataLoader(
    organ_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=0,  # you can try 2 or 4 later
)



Using 200 organ files from C:\Users\Admin\Desktop\ai\tone_shift\data\nsynth-organ\audio


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

K_H = 60   # harmonics_dim
K_N = 65   # noise_dim

decoder = TimbreDecoder(
    in_dim=3,
    harmonics_dim=K_H,
    noise_dim=K_N,
).to(device)

ms_loss = MultiScaleSpectrogramLoss().to(device)
optimizer = torch.optim.Adam(decoder.parameters(), lr=3e-4)

SR = 16000
HOP_LENGTH = 512
FRAME_RATE = SR / HOP_LENGTH


Using device: cuda


In [27]:
num_epochs = 10  # start small; you can increase later

for epoch in range(num_epochs):
    decoder.train()
    running_loss = 0.0
    n_batches = len(organ_loader)

    print(f"\n=== Epoch {epoch+1}/{num_epochs} ===")

    for i, batch in enumerate(organ_loader, start=1):
        feats = batch["feats"].to(device)    # [1, T, 3]
        f0    = batch["f0"].to(device)      # [1, T]
        audio = batch["audio"].to(device)   # [1, N]

        optimizer.zero_grad()

        # Forward through decoder
        harmonics, noise, gain, _ = decoder(feats)

        params = {
            "harmonic_amps": harmonics,
            "noise_amps": noise,
            "gain": gain,
            "optional": {}
        }

        # Synthesize – NO noise for now (forces model to use harmonics)
        y_hat = synthesize_from_params(
            params,
            f0,
            sr=SR,
            frame_rate=FRAME_RATE,
            noise_scale=0.0,
        )

        # Match lengths
        N = min(y_hat.shape[1], audio.shape[1])
        y_hat = y_hat[:, :N]
        y_ref = audio[:, :N]

        loss = ms_loss(y_hat, y_ref)

        if not torch.isfinite(loss):
            print(f"[Epoch {epoch+1} | step {i}/{n_batches}] non-finite loss, skipping batch")
            continue

        loss.backward()
        torch.nn.utils.clip_grad_norm_(decoder.parameters(), 1.0)
        optimizer.step()

        running_loss += loss.item()

        # 🧾 Логи кожного кроку
        wav_path = batch["path"][0] if isinstance(batch["path"], (list, tuple)) else batch["path"]
        print(
            f"[Epoch {epoch+1}/{num_epochs}] "
            f"step {i}/{n_batches} | loss={loss.item():.4f} | file={os.path.basename(wav_path)}",
            flush=True,
        )

    if n_batches > 0:
        mean_loss = running_loss / n_batches
        print(f"=== Epoch {epoch+1} finished | mean loss: {mean_loss:.4f} ===")



=== Epoch 1/10 ===
[Epoch 1/10] step 1/200 | loss=0.4032 | file=organ_electronic_058-042-050.wav
[Epoch 1/10] step 2/200 | loss=0.3085 | file=organ_electronic_015-039-050.wav
[Epoch 1/10] step 3/200 | loss=0.3510 | file=organ_electronic_026-055-075.wav
[Epoch 1/10] step 4/200 | loss=0.3239 | file=organ_electronic_034-027-127.wav
[Epoch 1/10] step 5/200 | loss=0.2192 | file=organ_electronic_033-065-025.wav
[Epoch 1/10] step 6/200 | loss=0.2462 | file=organ_electronic_000-070-127.wav
[Epoch 1/10] step 7/200 | loss=0.1831 | file=organ_acoustic_001-043-127.wav
[Epoch 1/10] step 8/200 | loss=0.1724 | file=organ_electronic_056-063-025.wav
[Epoch 1/10] step 9/200 | loss=0.1626 | file=organ_electronic_004-040-025.wav
[Epoch 1/10] step 10/200 | loss=0.2493 | file=organ_electronic_106-084-075.wav
[Epoch 1/10] step 11/200 | loss=0.2664 | file=organ_electronic_006-058-127.wav
[Epoch 1/10] step 12/200 | loss=0.1640 | file=organ_electronic_049-045-075.wav
[Epoch 1/10] step 13/200 | loss=0.1925 | fi

In [28]:
ckpt_dir = os.path.join(PROJECT_ROOT, "checkpoints")
os.makedirs(ckpt_dir, exist_ok=True)

ckpt_path = os.path.join(ckpt_dir, "decoder_organ_ddsp.pth")

torch.save(
    {
        "model_state_dict": decoder.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "config": {
            "in_dim": 3,
            "K_H": K_H,
            "K_N": K_N,
            "sr": SR,
            "hop_length": HOP_LENGTH,
        },
    },
    ckpt_path,
)

print("Saved organ decoder checkpoint to:", ckpt_path)


Saved organ decoder checkpoint to: C:\Users\Admin\Desktop\ai\tone_shift\checkpoints\decoder_organ_ddsp.pth


In [44]:
checkpoint = torch.load(ckpt_path, map_location="cpu")
decoder = TimbreDecoder(in_dim=3, harmonics_dim=K_H, noise_dim=K_N).to("cpu")
decoder.load_state_dict(checkpoint["model_state_dict"])
decoder.eval()


C:\Users\Admin\AppData\Local\Temp\ipykernel_15080\1352961866.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location="cpu")


TimbreDecoder(
  (gru): GRU(3, 256, num_layers=2, batch_first=True)
  (mlp): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=256, bias=True)
    (4): ReLU()
  )
  (harmonics_head): Linear(in_features=256, out_features=60, bias=True)
  (noise_head): Linear(in_features=256, out_features=65, bias=True)
  (gain_head): Linear(in_features=256, out_features=1, bias=True)
)

In [45]:
import os
import librosa
import soundfile as sf

SR = 16000  # or whatever you use in your project

def cut_wav_segment(
    in_path: str,
    start_sec: float,
    end_sec: float,
    sr: int = SR,
    mono: bool = True,
) -> str:
    """
    Cut a segment from a WAV file and save it as a new WAV.

    Args:
        in_path:   path to the original .wav
        start_sec: start time in seconds
        end_sec:   end time in seconds (if <= 0, uses full length)
        sr:        target sample rate (audio will be loaded/resampled to this)
        mono:      convert to mono

    Returns:
        out_path: path to the new cut .wav file
    """
    if end_sec > 0 and end_sec <= start_sec:
        raise ValueError("end_sec must be > start_sec (or <= 0 to mean 'till end').")

    # Load and (optionally) resample
    y, file_sr = librosa.load(in_path, sr=sr, mono=mono)

    n_samples = len(y)
    total_dur = n_samples / sr

    # Clamp times
    start_sec = max(0.0, min(start_sec, total_dur))
    if end_sec <= 0:
        end_sec = total_dur
    else:
        end_sec = max(start_sec, min(end_sec, total_dur))

    start_sample = int(start_sec * sr)
    end_sample   = int(end_sec * sr)

    y_seg = y[start_sample:end_sample]
    if y_seg.size == 0:
        raise ValueError("Cut segment is empty – check start_sec / end_sec.")

    # Build output path: same folder + suffix
    base, ext = os.path.splitext(in_path)
    out_path = f"{base}_cut_{start_sample}_{end_sample}{ext}"

    # Save segment
    sf.write(out_path, y_seg, sr)
    return out_path


In [41]:
orig_wav = r"C:\Users\Admin\Desktop\ai\tone_shift\document_5305266375659393036.wav"

# Cut from 1.0s to 2.5s and save as a new wav
seg_wav = cut_wav_segment(orig_wav, start_sec=0.0, end_sec=5.0)

In [48]:
from tone_shift.features.feature_extraction import extract_features  # якщо треба
from tone_shift.synth.synthesis import synthesize_from_params, make_wav

# 1) Вказуєш джерельний .wav
source_wav = r"C:\Users\Admin\Desktop\ai\tone_shift\instrumental-acoustic-guitar-music-401434.mp3"
# або будь-який інший файл

# 2) Готуєш фічі (та сама функція, що в тренуванні)
feats_t, f0_t, audio_t, frame_rate = prepare_features_for_decoder(source_wav)

feats_t = feats_t.to("cpu")   # [1, T, 3]
f0_t    = f0_t.to("cpu")      # [1, T]

print("feats_t:", feats_t.shape, "f0_t:", f0_t.shape)

# 3) Прогін через декодер + синтез
with torch.no_grad():
    harmonics, noise, gain, _ = decoder(feats_t)

    params = {
        "harmonic_amps": harmonics,
        "noise_amps": noise,
        "gain": gain,
        "optional": {}
    }

    # noise_scale=0.0 → чистий гармонічний орган
    # потім можеш пробувати 0.2–0.5 для трохи шумового характеру
    y_hat = synthesize_from_params(
        params,
        f0_t,
        sr=SR,
        frame_rate=FRAME_RATE,
        noise_scale=0.0,
    )

print("y_hat shape:", y_hat.shape)

# 4) Зберегти синтезований звук
make_wav(y_hat, SR)   # створить файл test_synth.wav у поточній директорії
print("✅ Saved synthesized audio to test_synth.wav")


feats_t: torch.Size([1, 7021, 3]) f0_t: torch.Size([1, 7021])
y_hat shape: torch.Size([1, 3594752])
✅ Audio saved to test_synth.wav
✅ Saved synthesized audio to test_synth.wav
y_hat shape: torch.Size([1, 3594752])
✅ Audio saved to test_synth.wav
✅ Saved synthesized audio to test_synth.wav
